In [2]:
import pandas as pd
from datetime import datetime
from common.bond_pricing import calc_bond_price_from_ytm, calc_bond_ytm_from_price, calc_bond_duration, estimate_years_to_maturity

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
df_vgsh = pd.read_excel(
    r"data/03-31-2024_VGSH_holdings_data_clean.xlsx"
)

In [11]:
def portfolio_analytics_df(df: pd.DataFrame):
    df["percentageOfFund"] = df["marketValue"] / df["marketValue"].sum()
    
    df["maturityDate"] = pd.to_datetime(df["maturityDate"], format="%m/%d/%Y")
    df["estimatedYearsToMaturity"] = df.apply(
        lambda row: estimate_years_to_maturity(
            maturity_date=row["maturityDate"], from_date=datetime(2024, 3, 31)
        ),
        axis=1,
    )
    df["currentYieldToMaturity"] = df.apply(
        lambda row: calc_bond_ytm_from_price(
            curr_bond_price=row["marketValue"],
            face_amount=row["faceAmount"],
            coupon=row["couponRate"],
            years_to_maturity=row["estimatedYearsToMaturity"],
        ),
        axis=1,
    )
    df["modifiedDuration"] = df.apply(
        lambda row: calc_bond_duration(
            row["currentYieldToMaturity"],
            row["couponRate"],
            row["estimatedYearsToMaturity"],
            face_amount=row["faceAmount"],
        )[0],
        axis=1,
    )
    df["priceFromPar"] = df.apply(
        lambda row: calc_bond_price_from_ytm(
            ytm=row["currentYieldToMaturity"],
            coupon=row["couponRate"],
            years_to_maturity=row["estimatedYearsToMaturity"],
        ),
        axis=1,
    )
    df["pv01"] = df["priceFromPar"] * df["modifiedDuration"] * 0.01
    df["dv01_from_pv01"] = df_vgsh["pv01"] * df["faceAmount"]
    df["dv01"] = df["marketValue"] * df["modifiedDuration"]
    portfolio_characteristics = {
        "number_of_holdings": len(df),
        "market_value": df["marketValue"].sum(), 
        "weighted_avg_coupon": (df["couponRate"] * df["marketValue"]).sum() / df["marketValue"].sum(),
        "weighted_avg_ytm": (df["currentYieldToMaturity"] * df["marketValue"]).sum() / df["marketValue"].sum(),
        "weighted_avg_maturity": (df["estimatedYearsToMaturity"] * df["marketValue"]).sum() / df["marketValue"].sum(),
        "weighted_avg_mod_duration": (df["modifiedDuration"] * df["marketValue"]).sum() / df["marketValue"].sum(),
        "portfolio_pv01": df["pv01"].sum(),
        "portfolio_dv01": df["dv01"].sum(),
    } 

    display(df)
    return portfolio_characteristics

In [12]:
portfolio_analytics_df(df=df_vgsh)

,type,asOfDate,longName,shortName,sharesHeld,marketValue,couponRate,maturityDate,faceAmount,ticker,...,cusip,sedol,percentageOfFund,estimatedYearsToMaturity,currentYieldToMaturity,modifiedDuration,priceFromPar,pv01,dv01_from_pv01,dv01
0,portfolioHolding,2024-03-31T00:00:00-04:00,United States Treasury Note/Bond,US TREASURY N/B,0,4.670116e+08,1.500,2027-01-31,506503000,TNOTE,...,912828Z78,BKMH699,0.019257,2.877778,4.849225,2.461221,92.203125,2.269322,1.149419e+09,1.149419e+09
1,portfolioHolding,2024-03-31T00:00:00-04:00,United States Treasury Note/Bond,US TREASURY N/B,0,3.859648e+08,0.500,2026-02-28,417612000,TNOTE,...,91282CBQ3,BLKH0X0,0.015915,1.941667,5.850496,1.496095,92.421875,1.382719,5.774401e+08,5.774401e+08
2,portfolioHolding,2024-03-31T00:00:00-04:00,United States Treasury Note/Bond,UST NOTE,0,3.852142e+08,2.250,2025-11-15,401069000,TNOTE,...,912828M56,BYQ26R6,0.015884,1.650000,5.018775,1.483000,96.046875,1.424376,5.712729e+08,5.712729e+08
3,portfolioHolding,2024-03-31T00:00:00-04:00,United States Treasury Note/Bond,US TREASURY N/B,0,3.763868e+08,0.750,2026-05-31,408561000,TNOTE,...,91282CCF6,BM9BQT8,0.015520,2.197222,4.933267,1.988272,92.125000,1.831696,7.483595e+08,7.483595e+08
4,portfolioHolding,2024-03-31T00:00:00-04:00,United States Treasury Note/Bond,US TREASURY N/B,0,3.719445e+08,0.375,2026-01-31,402306000,TNOTE,...,91282CBH3,BMZ2XK5,0.015337,1.863889,5.695486,1.497069,92.453125,1.384087,5.568267e+08,5.568267e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,portfolioHolding,2024-03-31T00:00:00-04:00,United States Treasury Note/Bond,US TREASURY N/B,0,8.182526e+07,1.125,2027-02-28,89887000,TNOTE,...,912828ZB9,BL1LY73,0.003374,2.955556,4.985166,2.470580,91.031250,2.249000,2.021558e+08,2.021558e+08
91,portfolioHolding,2024-03-31T00:00:00-04:00,United States Treasury Note/Bond,UST BOND,0,2.618800e+07,6.000,2026-02-15,25600000,TBOND,...,912810EW4,2888163,0.001080,1.905556,4.400873,1.457266,102.296875,1.490738,3.816289e+07,3.816289e+07
92,portfolioHolding,2024-03-31T00:00:00-04:00,United States Treasury Note/Bond,UST BOND,0,2.578368e+07,6.750,2026-08-15,24666000,TBOND,...,912810EX2,2942434,0.001063,2.408333,4.359579,1.906546,104.531250,1.992937,4.915778e+07,4.915778e+07
93,portfolioHolding,2024-03-31T00:00:00-04:00,United States Treasury Note/Bond,UST BOND,0,1.610667e+07,6.500,2026-11-15,15374000,TBOND,...,912810EY0,2941572,0.000664,2.663889,4.464223,2.351555,104.765625,2.463621,3.787572e+07,3.787572e+07


{'number_of_holdings': 95,
 'market_value': 24251480864.319996,
 'weighted_avg_coupon': 2.573554311542195,
 'weighted_avg_ytm': 5.004098030921241,
 'weighted_avg_maturity': 1.9989149843377658,
 'weighted_avg_mod_duration': 1.71197016800408,
 'portfolio_pv01': 155.12485054944165,
 'portfolio_dv01': 41517811769.637634}